#### Lecture 1. Graph Neural Network

1. 정점표현 학습
- 그래프에서 정점간 유사도를 임베딩 공간에서도 보존하는 것이 목적
- 출력으로 임베딩 자체를 얻는 변환식 임베딩 방법의 한계가 있음
    - 추가된 정점에 대해서는 임베딩을 얻을수 없음
    - 모든 정점에 대한 임베딩을 미리 계산하여 저장해두어야 함
- 귀납식 임베딩 방법 : ENV(v) = 그래프 구조와 정점의 부가 정보를 활용하는 복잡한 함수 => 그래프 신경망
    - 추가된 정점에 대해서도 임베딩을 얻을 수 있음
    - 모든 정점에 대해 미리 계산하여 저장해 둘 필요가 없음
    - 정점의 속성 정보 활용 가능

2. 그래프 신경망 기초
- 그래프 (인접 행렬 A로 표현) 와 정점의 속성정보(m차원 벡터, m은 속성의 수)
- 이웃 정점들의 정보를 집계하는 과정을 반복하여 임베딩을 얻음
- 0번 층, 입력 층의 임베딩으로는 정점의 속성 벡터를 사용하며 각 층에서는 이웃들의 이전층 임베딩을 집계함
- 대상 정점별 집계되는 구조 : 계산 그래프(computational graph)
- 다른 대상 정점간에도 층별 집계 함수는 공유
- 집계 함수 : 이웃들 정보의 평균을 계산하고(차원을 1로) 신경망에 적용
$$ h_v^k = \sigma (W_k \sum_{u \in N(v)} \frac{h^{k-1}_u}{|N(v)|} + B_k h^{k-1}_v ) , h_v^0 = X_v$$
- 정점 간 거리를 보존하는 것을 목표로 손실함수 결정
    - 인접성 기반 시 $ \mathcal{L} = \sum_{(u,v) \in V\times V}||z_u^T z_v - A_{u,v}||^2 $
- 후속과제의 손실함수를 이용한 종단종(end-to-end) 학습을 이용한 분류가 정확도가 대체로 높음
- 오차역전파를 통해 손실함수를 최소화
- 학습된 그래프 신경망을 추가된 정점 뿐만 아니라 새로운 그래프에 적용할 수 있음

3. 그래프 신경망 변형
- GCN (그래프 합성곱 신경망)
$$ h_v^k = \sigma (W_k \sum_{u \in N(v)\cup v} \frac{h^{k-1}_u}{|N(u)||N(v)|})$$
- GraphSAGE : 이웃들의 임베딩을 AGG를 이용해 합친후 자신의 임베딩과 연결
$$ h_v^k = \sigma ([W_k AGG(\{h_u^{k-1},\forall u \in N(v)\}), B_k h_v^{k-1}])$$
    - AGG 함수로는 mean, pool, lstm 사용 가능
    
4. 합성곱 신경망과의 비교
- 이웃의 정보를 집계하는 과정을 반복한다는 공통점
- 합성곱 신경망이 주로 쓰이는 이미지는 인접 픽셀이 유용한 정보를 담고 있는 가능성이 높음
- 그래프의 인접행렬에서의 인접 원소는 제한된 정보를 가지며, 행렬의 행과 열 순서는 임의로 결정됨

#### Lecture 2. 그래프 신경망(심화)

1. 그래프 신경망의 학습
- 비지도 학습, 지도 학습이 모두 가능
- 비지도 학습 : 정점간 거리를 보존 // 지도 학습 : 후속 과제의 손실함수에 따른 학습

2. 그래프 신경망의 어텐션
- 기본 그래프 신경망 : 이웃들의 정보를 동일한 가중치로 평균냄
- 그래프 합성곱 신경망 : 단순 연결성을 고려한 가중치로 평균냄
- 그래프 어텐션 신경망 : 가중치 자체도 학습함(self-attention이 사용됨)
    - 정점 i의 임베딩 hi 에서 신경망 W를 곱해 새로운 임베딩 얻기 $\tilde h_i = h_i W $
    - 두정점의 새로운 임베딩을 연결한 후 어텐션 계수 내적 $e_{ij} = a^T[concat(\tilde h_i, \tilde h_j)] $
    - softmax 적용 $ \alpha_{ij} = softmax_j(e_{ij})$
- 여러개의 어텐션을 동시에 학습한 뒤 결과를 연결하여 사용(multi-head attention)

3. 그래프 표현학습과 그래프 풀링
- 그래프 표현 학습(그래프 임베딩) : 그래프 전체를 벡터의 형태로 표현하는 것, 그래프 분류 등에 활용됨
- 그래프 풀링 : 정점 임베딩으로부터 그래프 임베딩을 얻는 것

4. 지나친 획일화 (over-smoothing) 문제
- 그래프 신경망의 층의 수가 증가하면서 정점의 임베딩이 서로 유사해지는 현상
- 작은 세상 효과에 의해 적은 수의 층으로도 다수의 정점에 영향을 받게됨
- 그래프 신경망의 층의 수를 늘렸을 때 후속과제에서 정확도가 감소하는 현상이 발생
    - JK(jumping knowledge network) : 모든 층의 임베딩을 함께 사용
    - APPNP : 0번째 층의 제외하고는 신경망 없이 집계 함수를 단순화
    
5. 그래프 데이터 증강
- 유사도가 높은 정점 간의 간선을 추가하는 방법이 제안됨
- 정점 분류의 정확도가 개선됨
    